In [9]:
# !pip install pandas openai langchain streamlit python-dotenv PyPDF2

In [19]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback

import os
import json
import pandas as pd
import traceback

from dotenv import load_dotenv

import PyPDF2

In [20]:
load_dotenv() #take environment variables from .env

True

In [21]:
KEY=os.getenv("OPENAI_API_KEY")

In [22]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

In [28]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [29]:
"""
number=5 
subject="data science"
tone="simple"
"""

'\nnumber=5 \nsubject="data science"\ntone="simple"\n'

In [30]:
TEMPLATE = '''
Text:{text}
You are an expert MCQ maker. Given the above text, your job is to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conformed to the text.
Make sure to format your response like RESPONSE_JSON below and use it as a guide. Ensure to make {number} MCQs.
### RESPONSE_JSON
{response_json}

'''

In [31]:
quiz_generator_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [32]:
quiz_chain = LLMChain(llm=llm, prompt=quiz_generator_prompt, output_key="quiz", verbose=True)

In [33]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [34]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [35]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [42]:
generate_evaluate_chain = SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], output_variables=["quiz", "review"], verbose=True)

In [43]:
data_file_path = "../data.txt"

In [44]:
with open(data_file_path, "r") as file:
    TEXT = file.read()

In [45]:
print(TEXT)

Generative AI model architectures and how they have evolved
Truly generative AI models—deep learning models that can autonomously create content on demand—have evolved over the last dozen years or so. The milestone model architectures during that period include

Variational autoencoders (VAEs), which drove breakthroughs in image recognition, natural language processing and anomaly detection.
 

Generative adversarial networks (GANs) and diffusion models, which improved the accuracy of previous applications and enabled some of the first AI solutions for photo-realistic image generation.
 

Transformers, the deep learning model architecture behind the foremost foundation models and generative AI solutions today.

Variational autoencoders (VAEs)
An autoencoder is a deep learning model comprising two connected neural networks: One that encodes (or compresses) a huge amount of unstructured, unlabeled training data into parameters, and another that decodes those parameters to reconstruct the

In [46]:
# Serilize the python dictionnary into a json-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [49]:
NUMBER=5 
SUBJECT="Generative AI model architectures and how they have evolved"
TONE="simple"

How to setup [Token Usage Tracking](https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking) in LangChain:

In [50]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Generative AI model architectures and how they have evolved
Truly generative AI models—deep learning models that can autonomously create content on demand—have evolved over the last dozen years or so. The milestone model architectures during that period include

Variational autoencoders (VAEs), which drove breakthroughs in image recognition, natural language processing and anomaly detection.
 

Generative adversarial networks (GANs) and diffusion models, which improved the accuracy of previous applications and enabled some of the first AI solutions for photo-realistic image generation.
 

Transformers, the deep learning model architecture behind the foremost foundation models and generative AI solutions today.

Variational autoencoders (VAEs)
An autoencoder is a deep learning model comprising two connected neural networks: One that encodes (or compresses) a huge amount of unstruc

In [51]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2333
Prompt Tokens:1734
Completion Tokens:599
Total Cost:0.0037990000000000003


In [52]:
response

{'text': "Generative AI model architectures and how they have evolved\nTruly generative AI models—deep learning models that can autonomously create content on demand—have evolved over the last dozen years or so. The milestone model architectures during that period include\n\nVariational autoencoders (VAEs), which drove breakthroughs in image recognition, natural language processing and anomaly detection.\n \n\nGenerative adversarial networks (GANs) and diffusion models, which improved the accuracy of previous applications and enabled some of the first AI solutions for photo-realistic image generation.\n \n\nTransformers, the deep learning model architecture behind the foremost foundation models and generative AI solutions today.\n\nVariational autoencoders (VAEs)\nAn autoencoder is a deep learning model comprising two connected neural networks: One that encodes (or compresses) a huge amount of unstructured, unlabeled training data into parameters, and another that decodes those paramet

In [59]:
quiz_str = response.get("quiz")

In [66]:
quiz_dict = json.loads(quiz_str)

In [67]:
quiz_dict

{'1': {'mcq': 'Which generative AI model architecture was introduced in 2013 and can encode data like an autoencoder but decode multiple new variations of the content?',
  'options': {'a': 'Generative adversarial networks (GANs)',
   'b': 'Transformers',
   'c': 'Variational autoencoders (VAEs)',
   'd': 'Diffusion models'},
  'correct': 'c'},
 '2': {'mcq': "Which generative AI model architecture works by first adding noise to the training data until it's random and unrecognizable, and then training the algorithm to iteratively diffuse the noise to reveal a desired output?",
  'options': {'a': 'Generative adversarial networks (GANs)',
   'b': 'Transformers',
   'c': 'Variational autoencoders (VAEs)',
   'd': 'Diffusion models'},
  'correct': 'd'},
 '3': {'mcq': "Which generative AI model architecture was first documented in a 2017 paper published by Ashish Vaswani and others, and is at the core of most of today's headline-making generative AI tools?",
  'options': {'a': 'Generative adv

In [61]:
quiz_table_data = []
for key, value in quiz_dict.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [62]:
quiz_table_data

[{'MCQ': 'Which generative AI model architecture was introduced in 2013 and can encode data like an autoencoder but decode multiple new variations of the content?',
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational autoencoders (VAEs) | d: Diffusion models',
  'Correct': 'c'},
 {'MCQ': "Which generative AI model architecture works by first adding noise to the training data until it's random and unrecognizable, and then training the algorithm to iteratively diffuse the noise to reveal a desired output?",
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational autoencoders (VAEs) | d: Diffusion models',
  'Correct': 'd'},
 {'MCQ': "Which generative AI model architecture was first documented in a 2017 paper published by Ashish Vaswani and others, and is at the core of most of today's headline-making generative AI tools?",
  'Choices': 'a: Generative adversarial networks (GANs) | b: Transformers | c: Variational au

In [65]:
df=pd.DataFrame(quiz_table_data)

In [64]:
df

,MCQ,Choices,Correct
0,Which generative AI model architecture was int...,a: Generative adversarial networks (GANs) | b:...,c
1,Which generative AI model architecture works b...,a: Generative adversarial networks (GANs) | b:...,d
2,Which generative AI model architecture was fir...,a: Generative adversarial networks (GANs) | b:...,b
3,Which generative AI model architecture excels ...,a: Generative adversarial networks (GANs) | b:...,b
4,Which generative AI model architecture is driv...,a: Generative adversarial networks (GANs) | b:...,d


In [69]:
df.to_csv("Generative_AI_Quiz.csv",index=False)

In [84]:
from datetime import datetime

datetime.now().strftime('%d-%m-%Y_%H:%M:%S')

'11-06-2024_17:02:40'